# Financial Modeling Prep API Exploration

This notebook explores the available indices and financial data from the FMP API.

**Note:** FMP transitioned from legacy (v3/v4) endpoints to **stable** endpoints as of August 31, 2025.

**Key endpoints we'll explore:**
- Available indices list
- VIX and volatility data  
- Major market indices (S&P 500, DJIA, NASDAQ)
- Economic indicators
- Credit spreads (via ETFs if CDS not available)


In [ ]:
import os
import requests
import pandas as pd
from pprint import pprint

# Load API key from environment
API_KEY = os.environ.get('FMP_API_KEY')
if not API_KEY:
    try:
        from dotenv import load_dotenv
        load_dotenv()
        API_KEY = os.environ.get('FMP_API_KEY')
    except ImportError:
        pass

if not API_KEY:
    print("⚠️  FMP_API_KEY not found. Enter below:")
    API_KEY = input("Enter your FMP API key: ").strip()

# NEW stable base URL (not v3/v4)
BASE_URL = "https://financialmodelingprep.com/stable"
print(f"✅ API Key loaded (ending in ...{API_KEY[-4:] if len(API_KEY) > 4 else '****'})")


✅ API Key loaded (ending in ...PJmv)


In [ ]:
def fmp_get(endpoint, params=None):
    """Helper function to make FMP API requests using STABLE endpoints."""
    url = f"{BASE_URL}/{endpoint}"
    params = params or {}
    params['apikey'] = API_KEY
    
    print(f"📡 Calling: {url}")
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        print(f"   ✅ Success: {len(data) if isinstance(data, list) else 'dict'} items")
        return data
    else:
        print(f"   ❌ Error {response.status_code}: {response.text[:300]}")
        return None


## 1. Available Indices

Get all indices available in FMP using the new stable endpoint.


In [ ]:
# Get all available indices (STABLE endpoint)
# Try different endpoint variations
endpoints_to_try = [
    "available_indexes",           # underscore version
    "available-indexes",           # dash version  
    "symbol/available_indexes",    # with symbol prefix
    "symbol/available-indexes",    # with symbol prefix, dash
]

available_indices = None
for endpoint in endpoints_to_try:
    result = fmp_get(endpoint)
    if result and len(result) > 0:
        available_indices = result
        print(f"✅ Working endpoint: {endpoint}")
        break

if available_indices:
    df_indices = pd.DataFrame(available_indices)
    print(f"\n📊 Total available indices: {len(df_indices)}")
    print("Columns:", df_indices.columns.tolist())
    display(df_indices)
else:
    df_indices = None
    print("❌ No working endpoint found for available indices")


📡 Calling: https://financialmodelingprep.com/stable/symbol/available-indexes
   ❌ Error 404: []
Failed to fetch available indices


In [ ]:
# Search for specific indices of interest
keywords = ['vix', 'volatility', 'cds', 'credit', 'spread', 'fear', 'treasury', 'yield', 'bond']

if df_indices is not None and not df_indices.empty:
    print("🔍 Searching for indices related to VIX, volatility, credit, spreads...\n")
    
    for kw in keywords:
        mask = df_indices.apply(lambda row: row.astype(str).str.lower().str.contains(kw).any(), axis=1)
        matches = df_indices[mask]
        if not matches.empty:
            print(f"\n📌 Matches for '{kw}':")
            display(matches)


## 2. Real-Time Index Quotes

Get real-time quotes for all major indices.


In [ ]:
# Get batch index quotes (all major indices)
index_quotes = fmp_get("batch-index-quotes")

if index_quotes:
    df_quotes = pd.DataFrame(index_quotes)
    print(f"\n📊 Real-time quotes for {len(df_quotes)} indices")
    print("Columns:", df_quotes.columns.tolist())
    display(df_quotes)
else:
    df_quotes = None


📡 Calling: https://financialmodelingprep.com/stable/batch-index-quotes
   ✅ Success: 196 items

📊 Real-time quotes for 196 indices
Columns: ['symbol', 'price', 'change', 'volume']


,symbol,price,change,volume
0,^DJBGIE,4288.78,21.56,0
1,XGIDA.IS,11549.91,-213.31,0
2,XUHIZ.IS,10396.70,-70.26,0
3,RTSI.ME,1077.78,22.46,0
4,^105833-USD-STRD,2367.03,8.91,0
...,...,...,...,...
191,XIN9.FGI,15111.42,-11.84,0
192,^NZ50,13489.15,56.95,0
193,XTRZM.IS,1706.94,-7.81,0
194,^JN0U.JO,6480.49,35.23,0


In [ ]:
# Search for VIX in the quotes
if df_quotes is not None and not df_quotes.empty:
    print("🔍 Looking for VIX and volatility-related indices in quotes:\n")
    
    # Search for VIX-related symbols
    vix_keywords = ['vix', 'vol', 'fear']
    for kw in vix_keywords:
        mask = df_quotes.apply(lambda row: row.astype(str).str.lower().str.contains(kw).any(), axis=1)
        matches = df_quotes[mask]
        if not matches.empty:
            print(f"📌 Matches for '{kw}':")
            display(matches)
    
    # Show all unique symbols for reference
    print(f"\n📋 All {len(df_quotes)} index symbols available:")
    print(df_quotes['symbol'].tolist())


🔍 Looking for VIX and volatility-related indices in quotes:

📌 Matches for 'vix':


,symbol,price,change,volume
10,^VIX1D,11.08,-3.64,0
48,^VVIX,90.34,-1.83,0
88,^VIX,16.35,-0.84,0
157,^VIX3M,19.64,-0.38,0
178,^VIX6M,22.10,-0.35,0


## 3. Historical Index Prices

Get historical data for specific indices.


In [ ]:
# Try getting historical data for key indices
# Try different endpoint patterns
test_symbols = [
    ("^GSPC", "S&P 500"),
    ("^DJI", "Dow Jones"),
    ("^VIX", "VIX Volatility"),
]

historical_endpoints = [
    "historical-price-eod/{symbol}",         # EOD prices
    "historical-price-full/{symbol}",        # Full historical
    "historical_price_eod/{symbol}",         # underscore version
]

for symbol, name in test_symbols:
    encoded = symbol.replace("^", "%5E")
    print(f"\n📈 Trying {name} ({symbol}):")
    
    for ep_template in historical_endpoints:
        endpoint = ep_template.format(symbol=encoded)
        historical = fmp_get(endpoint)
        
        if historical:
            if isinstance(historical, dict) and 'historical' in historical:
                df_hist = pd.DataFrame(historical['historical'][:5])
                print(f"   ✅ Endpoint: {ep_template}")
                display(df_hist)
                break
            elif isinstance(historical, list) and len(historical) > 0:
                df_hist = pd.DataFrame(historical[:5])
                print(f"   ✅ Endpoint: {ep_template}")
                display(df_hist)
                break


📡 Calling: https://financialmodelingprep.com/stable/historical-price-full/index/%5EGSPC
   ❌ Error 404: []
📡 Calling: https://financialmodelingprep.com/stable/historical-price-full/index/%5EDJI
   ❌ Error 404: []
📡 Calling: https://financialmodelingprep.com/stable/historical-price-full/index/%5EIXIC
   ❌ Error 404: []
📡 Calling: https://financialmodelingprep.com/stable/historical-price-full/index/%5EVIX
   ❌ Error 404: []


## 4. Other Available Symbols

Explore what other types of data are available (commodities, forex, ETFs).


In [ ]:
# Explore various symbol types available in stable API
endpoints_to_try = [
    ("symbol/available-commodities", "Commodities"),
    ("symbol/available-forex-currency-pairs", "Forex Pairs"),
    ("symbol/available-etfs", "ETFs"),
    ("symbol/available-cryptocurrencies", "Cryptocurrencies"),
]

for endpoint, name in endpoints_to_try:
    data = fmp_get(endpoint)
    if data:
        df = pd.DataFrame(data)
        print(f"\n📊 {name}: {len(df)} symbols")
        if not df.empty:
            display(df.head(10))


📡 Calling: https://financialmodelingprep.com/stable/symbol/available-commodities
   ❌ Error 404: []
📡 Calling: https://financialmodelingprep.com/stable/symbol/available-forex-currency-pairs
   ❌ Error 404: []
📡 Calling: https://financialmodelingprep.com/stable/symbol/available-etfs
   ❌ Error 404: []
📡 Calling: https://financialmodelingprep.com/stable/symbol/available-cryptocurrencies
   ❌ Error 404: []


## 5. Credit Spread Proxies (ETFs)

Since CDS spreads may not be directly available, we can use credit/bond ETFs as proxies.


In [ ]:
# Credit/Bond ETFs that can proxy for credit spreads
credit_etfs = [
    'HYG',   # iShares iBoxx High Yield Corporate Bond
    'LQD',   # iShares iBoxx Investment Grade Corporate Bond
    'JNK',   # SPDR Bloomberg High Yield Bond
    'VCIT',  # Vanguard Intermediate-Term Corporate Bond
    'VCSH',  # Vanguard Short-Term Corporate Bond
    'IGSB',  # iShares Short-Term Corporate Bond
    'TLT',   # iShares 20+ Year Treasury Bond
    'IEF',   # iShares 7-10 Year Treasury Bond
    'SHY',   # iShares 1-3 Year Treasury Bond
]

print("📊 Credit/Bond ETF Quotes (proxies for credit spreads):\n")

# Try batch quote endpoint
etf_quotes = fmp_get(f"batch-quote/{','.join(credit_etfs)}")

if etf_quotes:
    df_etf = pd.DataFrame(etf_quotes)
    display(df_etf)
else:
    print("Batch quote failed, trying individual quotes...")
    for etf in credit_etfs:
        quote = fmp_get(f"quote/{etf}")
        if quote:
            if isinstance(quote, list) and len(quote) > 0:
                q = quote[0]
                print(f"  {q.get('symbol', etf):6s}: ${q.get('price', 'N/A'):>8} | Chg: {q.get('changesPercentage', 0):+.2f}%")


📊 Credit/Bond ETF Quotes (proxies for credit spreads):

📡 Calling: https://financialmodelingprep.com/stable/batch-quote/HYG,LQD,JNK,VCIT,VCSH,IGSB,TLT,IEF,SHY
   ❌ Error 404: []
Batch quote failed, trying individual quotes...
📡 Calling: https://financialmodelingprep.com/stable/quote/HYG
   ❌ Error 404: []
📡 Calling: https://financialmodelingprep.com/stable/quote/LQD
   ❌ Error 404: []
📡 Calling: https://financialmodelingprep.com/stable/quote/JNK
   ❌ Error 404: []
📡 Calling: https://financialmodelingprep.com/stable/quote/VCIT
   ❌ Error 404: []
📡 Calling: https://financialmodelingprep.com/stable/quote/VCSH
   ❌ Error 404: []
📡 Calling: https://financialmodelingprep.com/stable/quote/IGSB
   ❌ Error 404: []
📡 Calling: https://financialmodelingprep.com/stable/quote/TLT
   ❌ Error 404: []
📡 Calling: https://financialmodelingprep.com/stable/quote/IEF
   ❌ Error 404: []
📡 Calling: https://financialmodelingprep.com/stable/quote/SHY
   ❌ Error 404: []


## 6. Economic Data Endpoints

Explore treasury rates and economic calendar.


In [ ]:
# Try economic data endpoints
econ_endpoints = [
    "treasury",
    "economic-calendar", 
    "sector-performance",
]

for endpoint in econ_endpoints:
    data = fmp_get(endpoint)
    if data:
        df = pd.DataFrame(data) if isinstance(data, list) else None
        print(f"\n📊 {endpoint}:")
        if df is not None and not df.empty:
            display(df.head(10))
        else:
            pprint(data[:3] if isinstance(data, list) else data)


📡 Calling: https://financialmodelingprep.com/stable/treasury
   ❌ Error 404: []
📡 Calling: https://financialmodelingprep.com/stable/economic-calendar
   ✅ Success: 8844 items

📊 economic-calendar:


,date,country,event,currency,previous,estimate,actual,change,impact,changePercentage,unit
0,2025-11-30 23:50:00,JP,Capital Expenditure YoY (Q3),JPY,7.6,5.9,NaN,NaN,Medium,0.000,%
1,2025-11-30 22:00:00,AU,Judo Bank Manufacturing PMI (Nov),AUD,51.6,51.6,NaN,NaN,Low,0.000,None
2,2025-11-30 22:00:00,AU,S&P Global Manufacturing PMI (Nov),AUD,49.7,51.6,NaN,NaN,High,0.000,None
3,2025-11-30 21:45:00,NZ,Building Permits (Oct),NZD,7.2,-5.0,NaN,NaN,Low,0.000,%
4,2025-11-30 14:00:00,SA,M3 Money Supply YoY,SAR,7.8,NaN,NaN,NaN,Low,0.000,%
5,2025-11-30 14:00:00,SA,M3 Money Supply YoY (Oct),SAR,7.8,NaN,6.9,-0.9,Low,-11.538,%
6,2025-11-30 14:00:00,SA,Bank Lending YoY (Oct),SAR,13.0,13.0,12.0,-1.0,Low,-7.692,%
7,2025-11-30 10:00:00,US,OPEC Meeting,USD,NaN,NaN,NaN,NaN,Medium,0.000,None
8,2025-11-30 09:30:00,ZA,Budget Balance (Oct),ZAR,NaN,NaN,NaN,NaN,Low,0.000,None
9,2025-11-30 09:00:00,RW,Producer Price Index MoM (Oct),RWF,2.5,0.7,-0.8,-3.3,Low,-132.000,%


📡 Calling: https://financialmodelingprep.com/stable/sector-performance
   ❌ Error 404: []


eanign 

In [11]:
print("=" * 60)
print("📋 SUMMARY: FMP Stable API - Your Subscription")
print("=" * 60)

print("""
✅ WORKING ENDPOINTS (confirmed):
  • /stable/batch-index-quotes   → 196 indices (incl. VIX)
  • /stable/economic-calendar    → 8,844 economic events

❌ NOT AVAILABLE (404 - may need higher tier):
  • /stable/symbol/available-indexes
  • /stable/historical-price-full/...
  • /stable/symbol/available-commodities
  • /stable/symbol/available-etfs
  • /stable/treasury
  • /stable/sector-performance
  • /stable/quote/{symbol}
  • /stable/batch-quote/{symbols}

📌 Key Data You CAN Access:
  ✅ VIX Index (^VIX) - via batch-index-quotes
  ✅ S&P 500 (^GSPC), Dow Jones (^DJI), NASDAQ (^IXIC)
  ✅ 196 global indices with real-time prices
  ✅ Economic calendar events (8,844 items)
  
⚠️  NOT AVAILABLE with current subscription:
  • Historical price data
  • Individual stock/ETF quotes
  • CDS spreads (not available in FMP)
  • Credit ETF quotes (HYG, LQD, etc.)
""")

# Show the VIX data we found
if df_quotes is not None:
    vix = df_quotes[df_quotes['symbol'].str.contains('VIX', case=False, na=False)]
    if not vix.empty:
        print("📊 VIX Data from batch-index-quotes:")
        display(vix)


📋 SUMMARY: Available Data from FMP Stable API

📌 Key Endpoints Tested:
  • /stable/symbol/available-indexes     - List all indices
  • /stable/batch-index-quotes           - Real-time index quotes
  • /stable/historical-price-full/index/ - Historical index prices
  • /stable/symbol/available-commodities - Commodities list
  • /stable/symbol/available-etfs        - ETFs list
  • /stable/treasury                     - Treasury rates
  • /stable/economic-calendar            - Economic events
  • /stable/sector-performance           - Sector returns

📌 Key Data for Macro Analysis:
  ✅ VIX Index (if in available-indexes)
  ✅ Major stock indices (S&P 500, DJIA, NASDAQ)
  ✅ Treasury rates
  ✅ Economic calendar
  ✅ Sector performance
  ✅ Credit ETFs (HYG, LQD, JNK) - proxy for credit spreads
  ⚠️  CDS spreads - not directly available, use credit ETFs

📌 Note: 
  - FMP migrated to 'stable' endpoints (not v3/v4)
  - Some endpoints may require specific subscription tiers



# Show ALL available indices from batch-index-quotes
if df_quotes is not None and not df_quotes.empty:
    print(f"📋 ALL {len(df_quotes)} INDICES AVAILABLE:\n")
    
    # Sort by symbol for easier reading
    df_sorted = df_quotes.sort_values('symbol')
    
    # Display in a nice format
    pd.set_option('display.max_rows', 200)
    display(df_sorted)
    
    # Also save to CSV for reference
    csv_path = "available_indices.csv"
    df_sorted.to_csv(csv_path, index=False)
    print(f"\n💾 Saved to {csv_path}")
